# Artificial Neural Networks And Deep Learning
# Homework 1: Image Classification
# Transfer Learning

## Students: 

*   Julián Jiménez: 10657117
*   Samuel Polo: 10670388



# **Our approach of the Task:**
# Initial approach

After improving our latest CNN model created from scratch, we decided to advance with the Transfer Learning approach, where we could use a successful pre-trained model in 2 ways: using fine tuning (using the whole model as it was designed for) or using only the pre-trained weights of the convolutional layers and create/train the fully connected layer .

We decided to not use fine tunning and only create and train the fully connected layers as the amount of images in our training dataset was not significant compared to the ImageNet dataset. Also this improved our training time significantly and allowed us to try more models.

Our initial thought was to use VGG16.
With this model, we initiated with 2 Fully connected Layers and without Early Stopping, getting a result of 0.86600. Then we improved the creation of the validation set using stratified sampling, achieving better results. We prepared a third execution with one less FC layer and Early Stopping activated getting a score of 0.88400.

We then proceeded to try the previous setup but with VGG19, only to find that the score decreased (0.86600). We think this is because VGG16 solves a simpler task comparing to the performance of VGG19. So, we moved back to VGG16 and then made the following tests:


*   Increased Data Augmentation (score: 0.89200)
*   L2 Regularization (score: 0.85800)
*   Data Augmentation Modification, Elu as activation function and One FC Layer (score: 0.85400)
*   Default augmentation (we had an error regarding the previous method, so we switched to the classic one) and Elu as activation function (score: 0.88200)
*   Previous one but with one more FC Layer (score: 0.87000)

We then proceeded to do one final test with VGG16 with Max Pooling and Batch Normalization as we thought it could improve the performance:

*   2 FC Layers and Batch Normalization (score: 0.86200)
*   Maxpooling, 1FC Layer and Batch normalization (score: 0.82000)

After all of these tests, our best score was 0.89200 in which we tried maximize the performance of VGG16. Nonetheless we knew that after 4 years of development, other technologies arised and then we decided to try with InceptionResNetV2. We chose it as in the keras documentation it achieved a high percentage accuracy in top-1 and top-5 prediction accuracy. This was also the case with NASNetLarge.

Our initial test showed considerable results getting us a score of 0.95600. We then proceeded to change the Activation function from ReLU to ELu and we increased the score to 0.96400. After multiple tests, changing the number of FC layers, applying Batch Normalization and Maxpooling we got up to 0.97000, in which after other several tests, we could only match our previous best score. 

Our final model used one fully connected layer elu activation function and dropout added.
The same results where achieved with ImageResNetV2 and NASNetLarge.




In [1]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

TensorFlow 2.x selected.
2.0.0


In [0]:
import numpy as np
import os

In [0]:
# Set the seed for random operations. 
# This allows our process to be reproducible. 

SEED = 1234
tf.random.set_seed(SEED)
np.random.seed(SEED)

# Get current working directory
cwd = os.getcwd()

In [0]:
#Set the directories for all, training and validation images.

dataset_dir = os.path.join(cwd, 'Classification_Dataset')
training_dir = os.path.join(dataset_dir,'training')
valid_dir = os.path.join(dataset_dir,'validation')

## Validation Directory Creation:

This code is used to create the validation directory and fill it up with images from all the classes.
The validation set is created with stratified sampling since all classes are not represented equally in the given dataset. In this way from each class we select the same proportion of images. This is done to reduce sampling error and allow all classes to be represented in the validation set.

In [0]:
#Creates validation directory if it does not exist already.
_ = os.makedirs(valid_dir, exist_ok=True)

In [0]:
#Proportion for Hold-out set validation.
train_valid_split = 0.2

#Fills the validtaion set 
subfolders = [f.path for f in os.scandir(training_dir) if f.is_dir() ] 

for subfold in subfolders:
  head_tail = os.path.split(subfold)
  _ = os.makedirs(os.path.join(valid_dir,head_tail[1]), exist_ok=True)
  list_files = [name for name in os.scandir(subfold) if os.path.isfile(name)]
  num_files = len(list_files)
  for num in np.random.choice(num_files, int(num_files*train_valid_split) ,replace=False):
    file_name = os.path.split(list_files[num])
    os.rename(list_files[num],os.path.join(valid_dir,head_tail[1],file_name[1]))

In [0]:
# Script for json creation

import json

dataset_split = {}
training_dic = {}
valid_dic = {}
for c in classes:
  training_dic[c] = []
  list_files = [name for name in os.scandir(os.path.join(training_dir,c)) if os.path.isfile(name)]
  for f in list_files:
    file_name = os.path.split(f)
    training_dic[c].append(file_name[1])
  valid_dic[c] = []
  list_files = [name for name in os.scandir(os.path.join(valid_dir,c)) if os.path.isfile(name)]
  for f in list_files:
    file_name = os.path.split(f)
    valid_dic[c].append(file_name[1])

dataset_split['training'] = training_dic
dataset_split['validation'] = valid_dic

with open('dataset_split.json', 'w') as fp:
     json.dump(dataset_split, fp)

## Training and Validation generators and datasets


In [0]:
 #ImageDataGenerator
 #------------------

from tensorflow.keras.preprocessing.image import ImageDataGenerator


#Create training ImageDataGenerator object
#Data augmentation is only applied to the training dataset

apply_data_augmentation = True

if apply_data_augmentation:
   train_data_gen = ImageDataGenerator(rotation_range=20,
                                       width_shift_range=0.2,
                                       height_shift_range=0.2,
                                       zoom_range=0.3,
                                       horizontal_flip=True,
                                       vertical_flip=True,
                                       fill_mode='constant',
                                       cval=0,
                                       rescale=1./255)
else:
  train_data_gen = ImageDataGenerator(rescale=1./255)
    
valid_data_gen = ImageDataGenerator(rescale=1./255)

In [0]:
# Create generators to read images from dataset directory
# -------------------------------------------------------

#Batch Size
bs = 64
# img shape
img_h = 331
img_w = 331

num_classes = 20

classes = ['owl',              # 0
           'galaxy',           # 1
           'lightning',        # 2
           'wine-bottle',      # 3
           't-shirt',          # 4
           'waterfall',        # 5
           'sword',            # 6
           'school-bus',       # 7
           'calculator',       # 8
           'sheet-music',      # 9
           'airplanes',        # 10
           'lightbulb',        # 11
           'skyscraper',       # 12
           'mountain-bike',    # 13
           'fireworks',        # 14
           'computer-monitor', # 15
           'bear',             # 16
           'grand-piano',      # 17
           'kangaroo',         # 18
           'laptop']           # 19

In [16]:
# Training generator
# Is shuffled
train_gen = train_data_gen.flow_from_directory(training_dir,
                                               batch_size=bs,
                                               classes=classes,
                                               class_mode='categorical',
                                               shuffle=True,
                                               target_size=(img_h, img_w),
                                               seed=SEED)

Found 1247 images belonging to 20 classes.


In [17]:
# Validation generator
# Is not shuffled
valid_gen = valid_data_gen.flow_from_directory(valid_dir,
                                               batch_size=bs,
                                               classes=classes,
                                               class_mode='categorical',
                                               shuffle=False,
                                               target_size=(img_h, img_w),
                                               seed=SEED)

Found 307 images belonging to 20 classes.


In [0]:
#Create Dataset objects
#Training
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
train_dataset = train_dataset.repeat()

In [0]:
# Validation 
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
valid_dataset = valid_dataset.repeat()

# Transfer Learning
## Best Choice: InceptionResNetV2/NASNetLarge


*   Other options explored: VGG16, VGG19

In [20]:
# Load NASNetLarge Model

nasnetL = tf.keras.applications.nasnet.NASNetLarge(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3),pooling='avg')

343613440/343610240 [==============================] - 10s 0us/step


In [21]:
# Visualize created model layers and parameters as a table

nasnetL.summary()

Model: "NASNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 331, 331, 3) 0                                            
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 165, 165, 96) 2592        input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 165, 165, 96) 384         stem_conv1[0][0]                 
__________________________________________________________________________________________________
activation (Activation)         (None, 165, 165, 96) 0           stem_bn1[0][0]                   
_____________________________________________________________________________________________

In [25]:
# Model Creation
# ------------

#We didn't use Finetuning because we were provided little training data 
#and the pre-trained models are expected to match this Classification problem.

irv2.trainable = False
    
model = tf.keras.Sequential()
model.add(nasnetL)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=4096, activation='elu'))
model.add(tf.keras.layers.Dropout(0.2,seed=SEED))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

# Visualize created model layers and parameters as a table
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
NASNet (Model)               (None, 4032)              84916818  
_________________________________________________________________
flatten_1 (Flatten)          (None, 4032)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 4096)              16519168  
_________________________________________________________________
dropout_1 (Dropout)          (None, 4096)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 20)                81940     
Total params: 101,517,926
Trainable params: 16,601,108
Non-trainable params: 84,916,818
_________________________________________________________________


## Definition of parameters for Training

In [0]:
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

## Training Procedure

In [27]:
from datetime import datetime

cwd = os.getcwd()

exps_dir = os.path.join(cwd, 'transfer_learning_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   save_weights_only=True)  # False to save the model directly
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)  # if 1 shows weights histograms
callbacks.append(tb_callback)

# Early Stopping
# Restores the best weights found during the training
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20,restore_best_weights=True)
    callbacks.append(es_callback)


model.fit(x=train_dataset,
          epochs=200, 
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen), 
          callbacks=callbacks)

Train for 20 steps, validate for 5 steps
Epoch 1/200
20/20 [==============================] - 85s 4s/step - loss: 1.8684 - accuracy: 0.6889 - val_loss: 0.3984 - val_accuracy: 0.9055
Epoch 2/200
20/20 [==============================] - 65s 3s/step - loss: 0.5289 - accuracy: 0.8885 - val_loss: 0.1755 - val_accuracy: 0.9479
Epoch 3/200
20/20 [==============================] - 62s 3s/step - loss: 0.3647 - accuracy: 0.9142 - val_loss: 0.1789 - val_accuracy: 0.9511
Epoch 4/200
20/20 [==============================] - 63s 3s/step - loss: 0.3096 - accuracy: 0.9254 - val_loss: 0.1668 - val_accuracy: 0.9609
Epoch 5/200
20/20 [==============================] - 63s 3s/step - loss: 0.2315 - accuracy: 0.9383 - val_loss: 0.0666 - val_accuracy: 0.9837
Epoch 6/200
20/20 [==============================] - 62s 3s/step - loss: 0.2384 - accuracy: 0.9463 - val_loss: 0.1547 - val_accuracy: 0.9544
Epoch 7/200
20/20 [==============================] - 62s 3s/step - loss: 0.2024 - accuracy: 0.9495 - val_loss: 0.

# Output of the Solution

## csv Creation

In [0]:
def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

## Test set label assignment

In [0]:
from PIL import Image
from skimage import transform
test_dir = os.path.join(dataset_dir, 'test')

image_filenames = next(os.walk(test_dir))[2]

results = {}
for image_name in image_filenames:
  image_dir = os.path.join(test_dir,image_name)
  img = Image.open(image_dir).convert('RGB')
  img_array = np.array(img)
  img_array = np.array(img_array).astype('float32')/255.
  img_array = transform.resize(img_array, (img_h, img_w, 3))
  img_array = np.expand_dims(img_array, 0)
  pred = model.predict(img_array)
  prediction = np.argmax(model.predict(img_array))   # predicted class
  results[image_name] = prediction

create_csv(results)